In [1]:
import tensorflow as tf

In [21]:
'''This example demonstrates the use of Convolution1D for text classification.
Gets to 0.89 test accuracy after 2 epochs.
90s/epoch on Intel i5 2.4Ghz CPU.
10s/epoch on Tesla K40 GPU.

功能:进行文本分类
特点:一维卷积层
速度:690s/epoch on my Mac OS

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 690s - loss: 0.2222 - acc: 0.9116 - val_loss: 0.2597 - val_acc: 0.8898
Epoch 2/2
24992/25000 [============================>.] - ETA: 0s - loss: 0.1884 - acc: 0.9253
'''

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Convolution1D, MaxPooling1D
from keras.datasets import imdb
from keras import backend as K

In [22]:
# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 2

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Loading data...
65544192/65552540 [============================>.] - ETA: 0s25000 train sequences
25000 test sequences
Pad sequences (samples x time)
X_train shape: (25000, 400)
X_test shape: (25000, 400)


In [23]:
print('Build model...')
model = Sequential()

# 添加嵌入层，并映射词汇索引和嵌入维度
# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen,
                    dropout=0.2))

# 添加卷积层
# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
# 使用最大化池化
# we use max pooling:
model.add(MaxPooling1D(pool_length=model.output_shape[1]))

# 平面化卷积层，添加压缩层
# We flatten the output of the conv layer,
# so that we can add a vanilla dense layer:
model.add(Flatten())

# 添加隐藏层
# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# 用单一但愿输出层实现壁垒化喜哥妹曲线
# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

Build model...


In [25]:
# 二进制交叉熵
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          validation_data=(X_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 690s - loss: 0.2222 - acc: 0.9116 - val_loss: 0.2597 - val_acc: 0.8898
Epoch 2/2
25000/25000 [==============================] - 1536s - loss: 0.1884 - acc: 0.9253 - val_loss: 0.2520 - val_acc: 0.8955
